<a href="https://colab.research.google.com/github/chantmk/NLP_2021/blob/main/HW10/HW10_BERT_finetuing_finished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  HW10: BERT fintuning. 

In this exercise, you are going to learn how to perform fine-tuning on a transformer-based model. First, we will provide a tutorial on fine-tuning the Large Movie Review Dataset (IMDB dataset) using distilBERT (https://arxiv.org/abs/1910.01108). After that, you have to complete the exercise by fine-tuning on the TRUE call-center dataset (HW5). This homework is based on the Hugging Face tutorial (https://huggingface.co/transformers/custom_datasets.html).

### 1. Install transformers library form Hugging Face

In [1]:
# !pip install torch==1.4.0
!pip install transformers
!pip install pythainlp
!pip install sentencepiece

     |████████████████████████████████| 2.0MB 19.4MB/s 
     |████████████████████████████████| 3.2MB 50.0MB/s 
     |████████████████████████████████| 890kB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c45c24a4d8edf8a6f94291574a78d8e2d31f7ccee8b1b2c9d353b42d3f0795ed
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 10.6MB 18.4MB/s 
     |████████████████████████████████| 747kB 40.6MB/s 
     |████████████████████████████████| 1.2MB 20.7MB/s 


### 2. Download Large Movie Review Dataset 

In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2021-03-31 08:47:12--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  18.2MB/s    in 9.9s    

2021-03-31 08:47:22 (8.14 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



### 3. Preprocess the dataset  
Large Movie Review Dataset  is a dataset for binary sentiment classification. The input of this dataset is a movie review with its sentiment as a ground truth

In [3]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [4]:
print("Unique label is {}, nb. of train data = {}, test_data = {}".format(np.unique(train_labels), len(train_texts), len(test_texts)))
for i in range(5):
  print("Data = {}".format(train_texts[i]))
  print("Label = {}".format(train_labels[i]))

Unique label is [0 1], nb. of train data = 20000, test_data = 25000
Data = "La Maman et la putain" is the beautifulest film of all time. And what's most moving about it may be the relation between reality and art the movie deals with, which is directly inspired by Proust's "A la Recherche du temps perdu".<br /><br />Indeed, "La Maman et la putain" and "In search of lost time" apparently tell the same story : the one of the failure of love, which repeats itself endlessly. The first woman's name is always Gilberte, and the second woman appears like a twisted and deformed double of Gilberte : Veronika is like a "whore Gilberte", beautiful like the night, whereas Gilberte was pure, and "beautiful like the day". After the failure of the first love, a second love begins, but this one is like already doomed by the first one. Veronika takes the place of Gilberte, in Alexandre's life and in the movie. She progressively eclipses her, first by time to time, Gilberte's still coming when Alexandre 

After the dataset is processed, we tokenize each input sentence. This tokenizer has a start token of '[CLS'] (id 101) and a seperator token '[SEP]' (id 102) at the end of each sentence. If the word is an Out-of-vocabulary word (OOV), the token id is 100. The tokenized output has the following format :

```python
{
  'input_ids': List[List[Int]]. List of tokenized input sentence.
  'attention_mask' : List[List[Int]].  List of masked token. See cell [7] for example.
}
```

In [5]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [6]:
tokenizer([ '[CLS] a' ], truncation=True, padding=True)

{'input_ids': [[101, 101, 1037, 102]], 'attention_mask': [[1, 1, 1, 1]]}

In [7]:
tokenizer( ['Pine apple apple pen  หมา ไก่', 'a b'], truncation=True, padding=True)

{'input_ids': [[101, 7222, 6207, 6207, 7279, 100, 100, 102], [101, 1037, 1038, 102, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0]]}

In [8]:
a = tokenizer(train_texts[:2], truncation=True, padding=True)
print(a)

{'input_ids': [[101, 1000, 2474, 9588, 2078, 3802, 2474, 2404, 8113, 1000, 2003, 1996, 3376, 4355, 2143, 1997, 2035, 2051, 1012, 1998, 2054, 1005, 1055, 2087, 3048, 2055, 2009, 2089, 2022, 1996, 7189, 2090, 4507, 1998, 2396, 1996, 3185, 9144, 2007, 1010, 2029, 2003, 3495, 4427, 2011, 4013, 19966, 1005, 1055, 1000, 1037, 2474, 28667, 5886, 5403, 4241, 29023, 2566, 8566, 1000, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 5262, 1010, 1000, 2474, 9588, 2078, 3802, 2474, 2404, 8113, 1000, 1998, 1000, 1999, 3945, 1997, 2439, 2051, 1000, 4593, 2425, 1996, 2168, 2466, 1024, 1996, 2028, 1997, 1996, 4945, 1997, 2293, 1010, 2029, 17993, 2993, 10866, 2135, 1012, 1996, 2034, 2450, 1005, 1055, 2171, 2003, 2467, 7664, 2063, 1010, 1998, 1996, 2117, 2450, 3544, 2066, 1037, 6389, 1998, 13366, 2953, 7583, 3313, 1997, 7664, 2063, 1024, 2310, 4948, 7556, 2003, 2066, 1037, 1000, 17219, 7664, 2063, 1000, 1010, 3376, 2066, 1996, 2305, 1010, 6168, 7664, 2063, 2001, 5760, 1010, 1998, 1000, 3376, 2066, 

In [9]:
train_encodings = tokenizer(train_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )
val_encodings = tokenizer(val_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )
test_encodings = tokenizer(test_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Convert the dataset into training format. You can see the training input format of distilBERT is in https://huggingface.co/transformers/model_doc/distilbert.html. 

In [10]:
train_data = [np.array(train_encodings['input_ids']), np.array(train_encodings['attention_mask'])]
val_data = [np.array(val_encodings['input_ids']), np.array(val_encodings['attention_mask'])]
test_data = [np.array(test_encodings['input_ids']), np.array(test_encodings['attention_mask'])]

### 4. Model fine-tuning
The model we used for fine-tuning is distilBERT (https://arxiv.org/abs/1910.01108), which is a smaller model distilled from the original BERT. Knowledge distillation is a well-known trick for improving the performance of a small model by learning an estimated uncertainty from a larger model instead of using a hard-label. If you want to know more about knowledge distillation, read https://arxiv.org/abs/1503.02531.

#### Model Initialization

In [11]:
from transformers import DistilBertForSequenceClassification
import torch

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels= 2)
model = torch.nn.DataParallel(model.cuda(), device_ids=[0])

LEARNING_RATE =  1e-5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

#### Set up training generator

In contrast to model.fit which you have used in the previous lab. A more common way to feed the data is to use a generator. It is more memory-efficient than model.fit as the data is only quired when the iterator executes. For example, you can set the generator to load the image from the folder when called instead of storing all of them in the RAM. An example below is a way to create a simple generator, which aggregate the data points into a batch. Both PyTorch and TensorFlow also has a utility module for creating a generator (torch.utils.data.DataLoader for Torch and tf.data.Dataset for Tensorflow) 

In [2]:
def batch_data_generator(data, label, bs = 8, training = True):
  while(True):
    X1= []
    X2 = []
    Y = []
    from sklearn.utils import shuffle
    ids, masks = data[0], data[1]
    if(training):
      ids, masks, label = shuffle(ids, masks, label, random_state = 42)
    for a, b, c in zip(ids, masks, label):
      X1.append(a)
      X2.append(b)
      Y.append(c)
      if(len(X1) == bs):
        yield [np.array(X1), np.array(X2)], np.array(Y)
        X1= []
        X2 = []
        Y = []
    if(len(X1) > 0):
      yield [np.array(X1), np.array(X2)], np.array(Y)
    if(not training):
      yield None
      break


In [13]:
train_generator = batch_data_generator(train_data, np.array(train_labels, dtype = np.int), training = True)

In [14]:
dummy_generator = batch_data_generator(train_data, np.array(train_labels, dtype = np.int), training = True)
X_dummy, Y_dummy = next(dummy_generator)
print(X_dummy[0].shape, X_dummy[1].shape, Y_dummy.shape)

(8, 512) (8, 512) (8,)


#### Start Fine-tuning

In [15]:
device = "cuda:0"
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
from collections import deque 

train_acc_stat =  deque(maxlen = 100)
train_loss_stat =  deque(maxlen = 100)

for step in  tqdm_notebook(range(1000)):
    X, Y = next(train_generator)
    ids = torch.tensor(X[0], dtype = torch.long, device = device)
    mask = torch.tensor(X[1], dtype = torch.long, device = device)
    targets = torch.tensor(Y, dtype = torch.long).to(device)

    optimizer.zero_grad()
    outputs = model(ids, mask)
    loss = loss_fn(outputs['logits'], targets)
    
    loss.backward()
    optimizer.step()

    with torch.no_grad():
      train_acc = accuracy_score(Y, outputs['logits'].argmax(axis = 1).cpu().detach().numpy() )
      train_loss = loss.cpu().detach().numpy()
      train_acc_stat.append(train_acc)
      train_loss_stat.append(train_loss)

    if (step + 1) %100==0:
      print("iter = {} train_acc = {}".format(step, np.array(train_acc_stat).mean()))
      print("iter = {} train_loss = {}".format(step, np.array(train_loss_stat).mean()))


    if (step + 1) %500==0:
      #validation step
      with torch.no_grad():
        val_generator = batch_data_generator(val_data, np.array(val_labels, dtype = np.int), training = False)
        y_true = []
        y_pred = []
        while(True):
          d = next(val_generator)
          if(d is None): break
          X, Y = d
          ids = torch.tensor(X[0], dtype = torch.long, device = device)
          mask = torch.tensor(X[1], dtype = torch.long, device = device)
          outputs_cls = model(ids, mask)['logits'].argmax(axis = 1).cpu().detach().numpy()
          y_true.append(Y)
          y_pred.append(outputs_cls)
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        print("val acc", accuracy_score(y_true, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


iter = 99 train_acc = 0.69375
iter = 99 train_loss = 0.5732057690620422
iter = 199 train_acc = 0.88625
iter = 199 train_loss = 0.2973712682723999
iter = 299 train_acc = 0.85
iter = 299 train_loss = 0.34989672899246216
iter = 399 train_acc = 0.8925
iter = 399 train_loss = 0.27587437629699707
iter = 499 train_acc = 0.90875
iter = 499 train_loss = 0.2356024533510208
val acc 0.8936
iter = 599 train_acc = 0.88625
iter = 599 train_loss = 0.28788113594055176
iter = 699 train_acc = 0.91375
iter = 699 train_loss = 0.23954825103282928
iter = 799 train_acc = 0.895
iter = 799 train_loss = 0.24813765287399292
iter = 899 train_acc = 0.905
iter = 899 train_loss = 0.2554687261581421
iter = 999 train_acc = 0.89125
iter = 999 train_loss = 0.2725937068462372
val acc 0.9106



## TODO 
Compare the classification performance between the non-transformer model and the model fine-tuned using pretrained WangchanBERTa on TRUE call-center dataset (HW5). WangchanBERTa (https://arxiv.org/abs/2101.09635) is RoBERTa (https://arxiv.org/abs/1907.11692) trained on thai texts. RoBERTa is also supported in Hugging Face (https://huggingface.co/transformers/model_doc/roberta.html).

To successfully fine-tune WangchanBERTa on the TRUE call-center dataset, you should:

1. Preprocess the dataset into the same format as the tutorial.
2. Tokenize the input from 1. See (https://colab.research.google.com/drive/1Kbk6sBspZLwcnOE61adAQo30xxqOQ9ko?usp=sharing&fbclid=IwAR23b8ZEoP6YxlUx7wWEu7dRCrVcyTFrZb3YSgI-nsxe_t4gy-bh8Rv5R9E#scrollTo=kAcpAdkddVQ8) for more details.
3. Process the tokenized input from 1. to the format that could be fed to the model.
4. Initialize WangchanBERTa (<b> you should choose the pretrained weight w.r.t. the tokenizer in 2.</b>)
5. Fine-tune the pretrained model.
6.  (Optional) Before fine-tuning is performed (before step 5), domain adaptation is often performed first by training a masked language model (maskLM). You can train maskLM by following this guideline (https://huggingface.co/transformers/model_doc/bert.html#bertformaskedlm).

### Import data

In [3]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2021-03-31 09:56:25--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv [following]
--2021-03-31 09:56:25--  https://www.dropbox.com/s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc19fb41b4f00583d03faba3cc10.dl.dropboxusercontent.com/cd/0/inline/BLtMTpVuBk2qNu2ZJp2pZaqTZkPLseKG4Bmo8oha_c3P4v9A4GrFQDDXcQ1K11NT40SMRx2oaa4rZe9KS3nCxEage5Wn7K3dPCCREDCZZ9ei5uGuM0tDfWj-Ja4z16mBYm3-QkF5fgQ_fEf7hWn8TBqU/file# [following]
--2021-03-31 09:56:26--  https://uc19fb41b4f00583d03faba3cc10.dl.dropboxusercontent.com/cd/0/inline/BLtMTpVuBk2qNu2ZJp2pZaqTZkP

In [5]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')
data_df

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues
...,...,...,...
16170,เชื่อมต่ออินเตอร์เน็ตไม่ได้ค่ะ,enquire,internet
16171,โทรออกต่างประเทศค่ะ,enquire,idd
16172,ยอดเงินเหลือเท่าไหร่ค่ะ,enquire,balance
16173,ยอดเงินในระบบ,enquire,balance


In [6]:
data_df.describe()

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


### Preprocess

In [7]:
def lowerString(dataframe, column):
    newColumn = column + "_clean"
    dataframe[newColumn] = dataframe[column].str.lower().copy()
    return dataframe

def label2num(dataframe, column):
    uniqueLabel = dataframe[column].unique()
    label2numMap = dict(zip(uniqueLabel, range(len(uniqueLabel))))
    num2labelMap = dict(zip(range(len(uniqueLabel)), uniqueLabel))
    dataframe[column+"_id"] = dataframe[column].map(label2numMap)
    return dataframe, label2numMap, num2labelMap
    
def getTextAndLabel(data):
    text = list(data["Sentence Utterance"])
    label = list(data["Object_clean_id"])
    return text, label

In [8]:
clean_df = data_df.copy()
clean_df = clean_df.applymap(lambda x: x.strip())
clean_df = lowerString(clean_df, "Action")
clean_df = lowerString(clean_df, "Object")
clean_df = clean_df.drop_duplicates("Sentence Utterance", keep="first")

In [9]:
map_df = clean_df.copy()
map_df, l2n_object, n2l_object = label2num(map_df, "Object_clean")

In [10]:
test_size = 0.2
val_size = 0.3
random_state = 2
train, test = train_test_split(map_df, test_size=test_size, random_state=random_state)
train, val = train_test_split(train, test_size=val_size, random_state=random_state)

In [11]:
train_texts, train_labels = getTextAndLabel(train)
val_texts, val_labels = getTextAndLabel(val)
test_texts, test_labels = getTextAndLabel(test)

In [12]:
print("Unique label is {}, nb. of train data = {}, test_data = {}".format(np.unique(train_labels), len(train_texts), len(test_texts)))
for i in range(5):
  print("Data = {}".format(train_texts[i]))
  print("Label = {}".format(train_labels[i]))

Unique label is [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25], nb. of train data = 7485, test_data = 2674
Data = ย้ายเครือข่าย
Label = 5
Data = ผมใช้ Iphone ผ่าน wifi router ที่บ้านนะครับ  มันเข้า ไอจูนไม่ได้ครับ
Label = 3
Data = จะเปิด โรมมิ่ง มือถือคะ
Label = 14
Data = ค่ะ สอบถามเรื่องโปรฯของ BB ค่ะ
Label = 11
Data = ผมอยากสอบถามยอดการใช้จ่ายเบอร์นี้ครับ
Label = 7


### Tokeniztion

In [13]:
model_name="wangchanberta-base-att-spm-uncased"
wangchan_tok = AutoTokenizer.from_pretrained(
                f'airesearch/{model_name}',
                revision='main',
                model_max_length=416,)

In [14]:
train_encodings = wangchan_tok(train_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )
val_encodings = wangchan_tok(val_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )
test_encodings = wangchan_tok(test_texts, add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,truncation=True
        )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [15]:
train_data = [np.array(train_encodings['input_ids']), np.array(train_encodings['attention_mask'])]
val_data = [np.array(val_encodings['input_ids']), np.array(val_encodings['attention_mask'])]
test_data = [np.array(test_encodings['input_ids']), np.array(test_encodings['attention_mask'])]

### Model

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(f'airesearch/{model_name}', num_labels=len(np.unique(train_labels)))
model = torch.nn.DataParallel(model.cuda(), device_ids=[0])

LEARNING_RATE =  1e-5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [17]:
train_generator = batch_data_generator(train_data, np.array(train_labels, dtype = np.int), training = True)

In [18]:
dummy_generator = batch_data_generator(train_data, np.array(train_labels, dtype = np.int), training = True)
X_dummy, Y_dummy = next(dummy_generator)
print(X_dummy[0].shape, X_dummy[1].shape, Y_dummy.shape)

(8, 512) (8, 512) (8,)


#### Start Fine-tuning

In [19]:
device = "cuda:0"
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
from collections import deque 

train_acc_stat =  deque(maxlen = 100)
train_loss_stat =  deque(maxlen = 100)

for step in  tqdm_notebook(range(1000)):
    X, Y = next(train_generator)
    ids = torch.tensor(X[0], dtype = torch.long, device = device)
    mask = torch.tensor(X[1], dtype = torch.long, device = device)
    targets = torch.tensor(Y, dtype = torch.long).to(device)

    optimizer.zero_grad()
    outputs = model(ids, mask)
    loss = loss_fn(outputs['logits'], targets)
    
    loss.backward()
    optimizer.step()

    with torch.no_grad():
      train_acc = accuracy_score(Y, outputs['logits'].argmax(axis = 1).cpu().detach().numpy() )
      train_loss = loss.cpu().detach().numpy()
      train_acc_stat.append(train_acc)
      train_loss_stat.append(train_loss)

    if (step + 1) %100==0:
      print("iter = {} train_acc = {}".format(step, np.array(train_acc_stat).mean()))
      print("iter = {} train_loss = {}".format(step, np.array(train_loss_stat).mean()))


    if (step + 1) %500==0:
      #validation step
      with torch.no_grad():
        val_generator = batch_data_generator(val_data, np.array(val_labels, dtype = np.int), training = False)
        y_true = []
        y_pred = []
        while(True):
          d = next(val_generator)
          if(d is None): break
          X, Y = d
          ids = torch.tensor(X[0], dtype = torch.long, device = device)
          mask = torch.tensor(X[1], dtype = torch.long, device = device)
          outputs_cls = model(ids, mask)['logits'].argmax(axis = 1).cpu().detach().numpy()
          y_true.append(Y)
          y_pred.append(outputs_cls)
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        print("val acc", accuracy_score(y_true, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


iter = 99 train_acc = 0.17125
iter = 99 train_loss = 2.8650588989257812
iter = 199 train_acc = 0.2725
iter = 199 train_loss = 2.545715570449829
iter = 299 train_acc = 0.36375
iter = 299 train_loss = 2.2805111408233643
iter = 399 train_acc = 0.4225
iter = 399 train_loss = 2.0565640926361084
iter = 499 train_acc = 0.48125
iter = 499 train_loss = 1.8693833351135254
val acc 0.5152743142144638
iter = 599 train_acc = 0.47125
iter = 599 train_loss = 1.8045213222503662
iter = 699 train_acc = 0.58125
iter = 699 train_loss = 1.4881559610366821
iter = 799 train_acc = 0.55
iter = 799 train_loss = 1.526831030845642
iter = 899 train_acc = 0.50125
iter = 899 train_loss = 1.6583350896835327
iter = 999 train_acc = 0.24275
iter = 999 train_loss = 2.6663076877593994
val acc 0.40523690773067333



In [22]:
with torch.no_grad():
    test_generator = batch_data_generator(test_data, np.array(test_labels, dtype = np.int), training = False)
    y_true = []
    y_pred = []
    while(True):
        d = next(test_generator)
        if(d is None): break
        X, Y = d
        ids = torch.tensor(X[0], dtype = torch.long, device = device)
        mask = torch.tensor(X[1], dtype = torch.long, device = device)
        outputs_cls = model(ids, mask)['logits'].argmax(axis = 1).cpu().detach().numpy()
        y_true.append(Y)
        y_pred.append(outputs_cls)
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    print("test acc", accuracy_score(y_true, y_pred))

test acc 0.4031413612565445
